In [1]:
import sys
sys.path.insert(0, "../..")

import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network
from module.simulation.meanfield import MeanField
import module.components.CONST as CONST

## Comparison of states and currents for larger systems
Four electrodes are attached to the flat system at all edges.

In [2]:
net_sizes = np.arange(2, 11)
nets = [Network(size, size, 1, [[0,0,0], [size - 1, 0, 0],[0, size - 1, 0],[size - 1, size - 1, 0]]) for size in net_sizes]
mfs = [MeanField(net) for net in nets]

In [3]:
voltage_configs = np.loadtxt("../../data/large_sys/voltage_configs.csv")

In [4]:
states_mf_e4 = []
for i in range(len(net_sizes)):
    size = net_sizes[i]
    state = np.loadtxt("../../data/large_sys/mf_1e-4/states"+str(size)+"x"+str(size)+".csv")
    states_mf_e4.append(state)
    
currents_mf_e4 = []
for i in range(len(net_sizes)):
    size = net_sizes[i]
    current = np.loadtxt("../../data/large_sys/mf_1e-4/currents"+str(size)+"x"+str(size)+".csv")
    currents_mf_e4.append(current)

In [5]:
states_mf_e3 = []
for i in range(len(net_sizes)):
    size = net_sizes[i]
    state = np.loadtxt("../../data/large_sys/mf_3e-3/states"+str(size)+"x"+str(size)+".csv")
    states_mf_e3.append(state)
    
currents_mf_e3 = []
for i in range(len(net_sizes)):
    size = net_sizes[i]
    current = np.loadtxt("../../data/large_sys/mf_3e-3/currents"+str(size)+"x"+str(size)+".csv")
    currents_mf_e3.append(current)

In [9]:
states_kmc = []
for i in range(len(net_sizes)):
    state = np.loadtxt("../../data/large_sys/kmc2/mean_state_Nx="+str(net_sizes[i])+"_Ny="+str(net_sizes[i])+"_Nz=1_Ne=4.csv", delimiter=",", skiprows=1)
    states_kmc.append(state)
    
currents_kmc = []
for i in range(len(net_sizes)):
    net_size = net_sizes[i]
    data = np.loadtxt("../../data/large_sys/kmc2/Nx="+str(net_size)+"_Ny="+str(net_size)+"_Nz=1_Ne=4.csv", delimiter = ",", skiprows = 1)
    currents_kmc.append(data)

In [10]:
voltages = voltage_configs[0]
net = nets[0]
mf = mfs[0]
net.set_voltage_config([voltages[0], voltages[1], voltages[2], voltages[3]], voltages[4])

In [11]:
for size_index in range(9):
    size = net_sizes[size_index]

    mf_sort_indices = np.argsort(voltage_configs[:,0], axis = 0)
    kmc_sort_indices = np.argsort(currents_kmc[size_index][:,0], axis = 0)

    sorted_mf_states = states_mf_e4[size_index][mf_sort_indices]
    sorted_mf_currents = currents_mf_e4[size_index][mf_sort_indices]

    sorted_voltage_config = voltage_configs[mf_sort_indices]

    # convergence metrics
    sorted_mf_convergence = []
    for i, state in enumerate(sorted_mf_states):
        nets[size_index].set_voltage_config([sorted_voltage_config[i, 0], sorted_voltage_config[i, 1], sorted_voltage_config[i, 2], sorted_voltage_config[i, 3]], sorted_voltage_config[i, 4])
        metric = mfs[size_index].convergence_metric((state - nets[size_index].dq) / CONST.electron_charge)
        sorted_mf_convergence.append(metric)
    sorted_mf_convergence = np.array(sorted_mf_convergence)
   

    sorted_kmc_states = states_kmc[size_index][kmc_sort_indices]
    sorted_kmc_currents = currents_kmc[size_index][kmc_sort_indices][:, 6] / 1000 * CONST.electron_charge
    sorted_kmc_errs = currents_kmc[size_index][kmc_sort_indices][:, 7] / 1000 * CONST.electron_charge
    sorted_kmc_steps = currents_kmc[size_index][kmc_sort_indices][:, 5]

    np.savetxt("../../data/large_sys/sorted/mf/mf_states"+str(size)+"x"+str(size)+".csv", sorted_mf_states)
    np.savetxt("../../data/large_sys/sorted/mf/mf_currents"+str(size)+"x"+str(size)+".csv", sorted_mf_currents)
    np.savetxt("../../data/large_sys/sorted/mf/mf_convergence"+str(size)+"x"+str(size)+".csv", sorted_mf_convergence)

    np.savetxt("../../data/large_sys/sorted/kmc/kmc_states"+str(size)+"x"+str(size)+".csv", sorted_kmc_states)
    np.savetxt("../../data/large_sys/sorted/kmc/kmc_currents"+str(size)+"x"+str(size)+".csv", sorted_kmc_currents)
    np.savetxt("../../data/large_sys/sorted/kmc/kmc_errs"+str(size)+"x"+str(size)+".csv", sorted_kmc_errs)
    np.savetxt("../../data/large_sys/sorted/kmc/kmc_steps"+str(size)+"x"+str(size)+".csv", sorted_kmc_steps)

    np.savetxt("../../data/large_sys/sorted/voltage_configs.csv", sorted_voltage_config)